In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Canny&Pyramid_L_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_f1 = np.load('../trainImage_canny.npy')
        self.test_x_f1 = np.load('../testImage_canny.npy')
        self.train_x_f2 = np.load('../trainImage_pyramid_L.npy')/255.0
        self.test_x_f2 = np.load('../testImage_pyramid_L.npy')/255.0
        self.train_x_f3 = np.load('../trainImage_pyramid_G.npy')/255.0
        self.test_x_f3 = np.load('../testImage_pyramid_G.npy')/255.0

        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_f1, x_f2, x_f3, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_f1",x_f1.get_shape()) 
            print("CNN:x_f2",x_f2.get_shape())  
            print("CNN:x_f3",x_f3.get_shape())  
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x_f1,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_f1,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())
            
            #输入x_f2,卷积核为3*3 输出维度为32
            net1_3 = tf.layers.conv2d(inputs = x_f2,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_3'               # 命名用于获取变量
                                    )
            print("CNN:",net1_3.get_shape())
            
            #输入x_f3,卷积核为3*3 输出维度为32
            net1_4 = tf.layers.conv2d(inputs = x_f3,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_4'               # 命名用于获取变量
                                    )
            print("CNN:",net1_4.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.concat([net, net1_3], 3)
            net = tf.concat([net, net1_4], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_f1 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f1.shape[1], 
                                                     self.train_x_f1.shape[2], self.train_x_f1.shape[3]],
                                  name='x_f1')
        self.x_f2 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f2.shape[1], 
                                                     self.train_x_f2.shape[2], self.train_x_f2.shape[3]],
                                  name='x_f2')
        self.x_f3 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f3.shape[1], 
                                             self.train_x_f3.shape[2], self.train_x_f3.shape[3]],
                          name='x_f3')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_f1, self.x_f2, self.x_f3, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x,  self.x_f1, self.x_f2, self.x_f3, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_f1 = self.train_x_f1[shuffle_idxs]
            shuffled_set_f2 = self.train_x_f2[shuffle_idxs]
            shuffled_set_f3 = self.train_x_f3[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f1 = shuffled_set_f1[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f2 = shuffled_set_f2[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f3 = shuffled_set_f3[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_f1: batch_x_f1,
                                                          self.x_f2: batch_x_f2,
                                                          self.x_f3: batch_x_f3,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f3_test =self.test_x_f3[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_f1: batch_x_f1_test,
                                                                    self.x_f2: batch_x_f2_test,
                                                                    self.x_f3: batch_x_f3_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f3_test =self.test_x_f3[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.x_f3: batch_x_f3_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [ ]:
dataset = '4_Flowers_1s'
epoch = 80
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
# Avg per epoch ptime: 10.01, total 80 epochs ptime: 846.76
#  [*] Training finished!
#  [*] Best Epoch:  20 , Accuracy:  0.7599999308586121
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay-20
#  [*] Finished testing Best Epoch: 20 , accuracy:  0.7599999308586121 !

CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN:x_f3 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 128)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN:x_f3 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 128)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [

Epoch: [ 5] [   8/  20] time: 48.6693, loss: 0.00767415
Epoch: [ 5] [   9/  20] time: 49.1230, loss: 0.02674006
Epoch: [ 5] [  10/  20] time: 49.5887, loss: 0.08388700
Epoch: [ 5] [  11/  20] time: 50.0600, loss: 0.01122781
Epoch: [ 5] [  12/  20] time: 50.5426, loss: 0.02599703
Epoch: [ 5] [  13/  20] time: 51.0020, loss: 0.01488030
Epoch: [ 5] [  14/  20] time: 51.4533, loss: 0.02967080
Epoch: [ 5] [  15/  20] time: 51.9054, loss: 0.02119436
Epoch: [ 5] [  16/  20] time: 52.3562, loss: 0.04280825
Epoch: [ 5] [  17/  20] time: 52.8200, loss: 0.02671346
Epoch: [ 5] [  18/  20] time: 53.2744, loss: 0.00978928
Epoch: [ 5] [  19/  20] time: 53.7454, loss: 0.02139311
[5/80] - ptime: 10.0304 loss: 0.02397736 acc: 0.21000 lr: 0.00100000
Epoch: [ 6] [   0/  20] time: 55.5534, loss: 0.00454256
Epoch: [ 6] [   1/  20] time: 56.0084, loss: 0.01128612
Epoch: [ 6] [   2/  20] time: 56.4549, loss: 0.02369104
Epoch: [ 6] [   3/  20] time: 56.9182, loss: 0.00652893
Epoch: [ 6] [   4/  20] time: 57.37

Epoch: [12] [   5/  20] time: 121.7333, loss: 0.01029557
Epoch: [12] [   6/  20] time: 122.1967, loss: 0.00768825
Epoch: [12] [   7/  20] time: 122.6453, loss: 0.02453582
Epoch: [12] [   8/  20] time: 123.1025, loss: 0.00617231
Epoch: [12] [   9/  20] time: 123.5567, loss: 0.01110442
Epoch: [12] [  10/  20] time: 124.0093, loss: 0.01502557
Epoch: [12] [  11/  20] time: 124.4633, loss: 0.00216306
Epoch: [12] [  12/  20] time: 124.9167, loss: 0.00807695
Epoch: [12] [  13/  20] time: 125.3789, loss: 0.03617557
Epoch: [12] [  14/  20] time: 125.8328, loss: 0.00485286
Epoch: [12] [  15/  20] time: 126.2905, loss: 0.01161657
Epoch: [12] [  16/  20] time: 126.7456, loss: 0.02061384
Epoch: [12] [  17/  20] time: 127.2043, loss: 0.00103490
Epoch: [12] [  18/  20] time: 127.6529, loss: 0.02339260
Epoch: [12] [  19/  20] time: 128.1071, loss: 0.08686573
[12/80] - ptime: 10.0363 loss: 0.01655384 acc: 0.49000 lr: 0.00090000
Epoch: [13] [   0/  20] time: 129.9271, loss: 0.01482668
Epoch: [13] [   1/

Epoch: [19] [   1/  20] time: 193.9902, loss: 0.01196761
Epoch: [19] [   2/  20] time: 194.4455, loss: 0.00318387
Epoch: [19] [   3/  20] time: 194.8982, loss: 0.00164215
Epoch: [19] [   4/  20] time: 195.3517, loss: 0.00269081
Epoch: [19] [   5/  20] time: 195.8155, loss: 0.01961377
Epoch: [19] [   6/  20] time: 196.2729, loss: 0.00421581
Epoch: [19] [   7/  20] time: 196.7342, loss: 0.00134254
Epoch: [19] [   8/  20] time: 197.1864, loss: 0.00902102
Epoch: [19] [   9/  20] time: 197.6519, loss: 0.00381451
Epoch: [19] [  10/  20] time: 198.1047, loss: 0.00539623
Epoch: [19] [  11/  20] time: 198.5621, loss: 0.00322905
Epoch: [19] [  12/  20] time: 199.0096, loss: 0.00144389
Epoch: [19] [  13/  20] time: 199.4537, loss: 0.00466176
Epoch: [19] [  14/  20] time: 199.9159, loss: 0.00306119
Epoch: [19] [  15/  20] time: 200.3647, loss: 0.00119737
Epoch: [19] [  16/  20] time: 200.8195, loss: 0.08885644
Epoch: [19] [  17/  20] time: 201.2723, loss: 0.01403618
Epoch: [19] [  18/  20] time: 2

Epoch: [25] [  18/  20] time: 265.1252, loss: 0.00024564
Epoch: [25] [  19/  20] time: 265.5786, loss: 0.00097133
[25/80] - ptime: 10.0530 loss: 0.00033925 acc: 0.58000 lr: 0.00081000
Epoch: [26] [   0/  20] time: 267.4203, loss: 0.00029303
Epoch: [26] [   1/  20] time: 267.8835, loss: 0.00003664
Epoch: [26] [   2/  20] time: 268.3362, loss: 0.00001343
Epoch: [26] [   3/  20] time: 268.8020, loss: 0.00000302
Epoch: [26] [   4/  20] time: 269.2587, loss: 0.00008021
Epoch: [26] [   5/  20] time: 269.7136, loss: 0.00020175
Epoch: [26] [   6/  20] time: 270.1688, loss: 0.00011083
Epoch: [26] [   7/  20] time: 270.6198, loss: 0.00036555
Epoch: [26] [   8/  20] time: 271.0825, loss: 0.00003301
Epoch: [26] [   9/  20] time: 271.5326, loss: 0.00000757
Epoch: [26] [  10/  20] time: 271.9964, loss: 0.00014551
Epoch: [26] [  11/  20] time: 272.4521, loss: 0.00000289
Epoch: [26] [  12/  20] time: 272.9155, loss: 0.00170044
Epoch: [26] [  13/  20] time: 273.3681, loss: 0.00017934
Epoch: [26] [  14/

...

Epoch: [53] [   2/  20] time: 553.5475, loss: 0.00000014
Epoch: [53] [   3/  20] time: 554.0003, loss: 0.00002628
Epoch: [53] [   4/  20] time: 554.4715, loss: 0.00003026
Epoch: [53] [   5/  20] time: 554.9244, loss: 0.00000169
Epoch: [53] [   6/  20] time: 555.3734, loss: 0.00000030
Epoch: [53] [   7/  20] time: 555.8228, loss: 0.00000081
Epoch: [53] [   8/  20] time: 556.2756, loss: 0.00000713
Epoch: [53] [   9/  20] time: 556.7405, loss: 0.00019718
Epoch: [53] [  10/  20] time: 557.1958, loss: 0.00005313
Epoch: [53] [  11/  20] time: 557.6573, loss: 0.00000595
Epoch: [53] [  12/  20] time: 558.1062, loss: 0.00000176
Epoch: [53] [  13/  20] time: 558.5670, loss: 0.00003063
Epoch: [53] [  14/  20] time: 559.0175, loss: 0.00000414
Epoch: [53] [  15/  20] time: 559.4765, loss: 0.00000517
Epoch: [53] [  16/  20] time: 559.9261, loss: 0.00000117
Epoch: [53] [  17/  20] time: 560.3793, loss: 0.00002033
Epoch: [53] [  18/  20] time: 560.8337, loss: 0.00000943
Epoch: [53] [  19/  20] time: 5

Epoch: [59] [  19/  20] time: 624.5792, loss: 0.00001268
[59/80] - ptime: 10.0321 loss: 0.00001004 acc: 0.61000 lr: 0.00059049
Epoch: [60] [   0/  20] time: 626.3889, loss: 0.00000666
Epoch: [60] [   1/  20] time: 626.8406, loss: 0.00000164
Epoch: [60] [   2/  20] time: 627.2901, loss: 0.00000268
Epoch: [60] [   3/  20] time: 627.7498, loss: 0.00000745
Epoch: [60] [   4/  20] time: 628.2009, loss: 0.00000723
Epoch: [60] [   5/  20] time: 628.6669, loss: 0.00000098
Epoch: [60] [   6/  20] time: 629.1195, loss: 0.00000204
Epoch: [60] [   7/  20] time: 629.5814, loss: 0.00006519
Epoch: [60] [   8/  20] time: 630.0347, loss: 0.00000005
Epoch: [60] [   9/  20] time: 630.4993, loss: 0.00000770
Epoch: [60] [  10/  20] time: 630.9490, loss: 0.00001473
Epoch: [60] [  11/  20] time: 631.4044, loss: 0.00000820
Epoch: [60] [  12/  20] time: 631.8671, loss: 0.00000111
Epoch: [60] [  13/  20] time: 632.3169, loss: 0.00000040
Epoch: [60] [  14/  20] time: 632.7763, loss: 0.00000011
Epoch: [60] [  15/

Epoch: [66] [  15/  20] time: 696.6582, loss: 0.00000200
Epoch: [66] [  16/  20] time: 697.1142, loss: 0.00000797
Epoch: [66] [  17/  20] time: 697.5760, loss: 0.00001447
Epoch: [66] [  18/  20] time: 698.0295, loss: 0.00002235
Epoch: [66] [  19/  20] time: 698.5028, loss: 0.00000048
[66/80] - ptime: 10.0924 loss: 0.00000592 acc: 0.61000 lr: 0.00053144
Epoch: [67] [   0/  20] time: 700.3730, loss: 0.00000147
Epoch: [67] [   1/  20] time: 700.8311, loss: 0.00001500
Epoch: [67] [   2/  20] time: 701.2880, loss: 0.00001465
Epoch: [67] [   3/  20] time: 701.7455, loss: 0.00000029
Epoch: [67] [   4/  20] time: 702.1947, loss: 0.00000030
Epoch: [67] [   5/  20] time: 702.6568, loss: 0.00001618
Epoch: [67] [   6/  20] time: 703.1092, loss: 0.00000438
Epoch: [67] [   7/  20] time: 703.5598, loss: 0.00000014
Epoch: [67] [   8/  20] time: 704.0194, loss: 0.00000687
Epoch: [67] [   9/  20] time: 704.4698, loss: 0.00000045
Epoch: [67] [  10/  20] time: 704.9342, loss: 0.00000453
Epoch: [67] [  11/

Epoch: [73] [  11/  20] time: 768.8083, loss: 0.00002828
Epoch: [73] [  12/  20] time: 769.2597, loss: 0.00000083
Epoch: [73] [  13/  20] time: 769.7197, loss: 0.00000209
Epoch: [73] [  14/  20] time: 770.1709, loss: 0.00000272
Epoch: [73] [  15/  20] time: 770.6219, loss: 0.00001259
Epoch: [73] [  16/  20] time: 771.0798, loss: 0.00000118
Epoch: [73] [  17/  20] time: 771.5316, loss: 0.00000293
Epoch: [73] [  18/  20] time: 771.9905, loss: 0.00000136
Epoch: [73] [  19/  20] time: 772.4431, loss: 0.00000103
[73/80] - ptime: 9.9615 loss: 0.00000726 acc: 0.61000 lr: 0.00047830
Epoch: [74] [   0/  20] time: 774.3156, loss: 0.00000013
Epoch: [74] [   1/  20] time: 774.7732, loss: 0.00000381
Epoch: [74] [   2/  20] time: 775.2281, loss: 0.00001358
Epoch: [74] [   3/  20] time: 775.6797, loss: 0.00000139
Epoch: [74] [   4/  20] time: 776.1520, loss: 0.00000081
Epoch: [74] [   5/  20] time: 776.5940, loss: 0.00000020
Epoch: [74] [   6/  20] time: 777.0611, loss: 0.00046914
Epoch: [74] [   7/ 

Epoch: [80] [   7/  20] time: 840.7290, loss: 0.00000088
Epoch: [80] [   8/  20] time: 841.1931, loss: 0.00000080
Epoch: [80] [   9/  20] time: 841.6454, loss: 0.00000040
Epoch: [80] [  10/  20] time: 842.0989, loss: 0.00006491
Epoch: [80] [  11/  20] time: 842.5536, loss: 0.00000319
Epoch: [80] [  12/  20] time: 843.0058, loss: 0.00000108
Epoch: [80] [  13/  20] time: 843.4735, loss: 0.00000293
Epoch: [80] [  14/  20] time: 843.9222, loss: 0.00052794
Epoch: [80] [  15/  20] time: 844.3834, loss: 0.00000034
Epoch: [80] [  16/  20] time: 844.8333, loss: 0.00000040
Epoch: [80] [  17/  20] time: 845.2968, loss: 0.00001051
Epoch: [80] [  18/  20] time: 845.7475, loss: 0.00000159
Epoch: [80] [  19/  20] time: 846.2060, loss: 0.00002755
[80/80] - ptime: 10.0123 loss: 0.00003388 acc: 0.61000 lr: 0.00043047
Avg per epoch ptime: 10.01, total 80 epochs ptime: 846.76
 [*] Training finished!
 [*] Best Epoch:  20 , Accuracy:  0.7599999308586121
INFO:tensorflow:Restoring parameters from checkpoint/C

In [ ]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放